# Data Preparation

## Load Constants

#### Create a file called constants.py

In [1]:
RAG_PROMPT = """You are an AI conversational assistant. Given the following context and question, provide a relevant and accurate answer. If you don't know the answer, just say that you don't know. Use only 50 words or less. \n\n
Be precise and accurate. Make sure you use proper grammar and that you answer the question taking into account the entire context. \n\n
Context:
{context}

Question:
{question}

Answer:"""

GRADER_PROMPT = """You are a grader assessing relevance of a retrieved document to a user question. \n 
Here is the retrieved document: \n\n {context} \n\n
Here is the user question: {question} \n
If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""

AGENT_PROMPT = """
You are an AI assistant. You first need to decide if you need to use the users past interactions. \n
You have the task to generate a new question based on the user's latest question or leave the question as it is. \n
You might need to rewrite the question to make it semantically similar to the user's previous interaction. \n
If the user asks to try again reform the last interaction. \n
Interactions are in the form of question-answer. \n
Here are the user's past interactions:
\n ------- \n
{past_interactions}

Now, the latest question is:
\n ------- \n
{user_question}
"""

REWRITE_QUERY = """ \n 
Look at the input and try to reason about the underlying semantic intent / meaning. \n 
Here is the initial question:
\n ------- \n
{question} 
\n ------- \n
Formulate an improved question: 
Only provide the question. Do not include any other information. \n"""

REWRITE_DECISION = "rewrite"
GENERATE_DECISION = "generate"

## Load Data (you can use Unstructured or Docling Loader for PDFs, Word, Excel, CSV)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url, verify_ssl = False).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

## Splitting and Indexing

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Vectorize the documents

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=hf,
    persist_directory='embd'
)

retriever = vectorstore.as_retriever(search_kwargs={'k': 2})


# Agentic Structure and Preparation

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.",
)

tools = [retriever_tool]

In [ ]:
from langchain_groq import ChatGroq

GROQ_API_KEY="your_api_key_here"
llm = ChatGroq(temperature=0, model="mixtral-8x7b-32768", api_key=GROQ_API_KEY, streaming=True, max_tokens=100)   


In [ ]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage

from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    user_question: str

In [ ]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

from pydantic import BaseModel, Field

from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
import constants
# use this to reload the constants module only for notebooks
import importlib
importlib.reload(constants)

checkpointer = MemorySaver()

### Edges


def grade_documents(state) -> Literal["generate", "rewrite"]:
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """

    print("---CHECK RELEVANCE---")

    # Data model
    class grade(BaseModel):
        """Binary score for relevance check."""

        binary_score: str = Field(description="Relevance score 'yes' or 'no'")

    # LLM
    model = llm

    # LLM with tool and validation
    llm_with_tool = model.with_structured_output(grade)

    messages = state["messages"]
    last_message = messages[-1]

    question = state["user_question"]
    docs = last_message.content
    # .format(question=question, context=docs)
    # Prompt
    prompt = PromptTemplate(
        template= constants.GRADER_PROMPT,
        input_variables=["context", "question"],
    )
    # Chain
    chain = prompt | llm_with_tool
    scored_result = chain.invoke({"question": question, "context": docs})

    score = scored_result.binary_score

    if score == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return constants.GENERATE_DECISION

    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(score)
        return constants.REWRITE_DECISION


### Nodes


def agent(state):
    print("---CALL AGENT---")
    messages = state["messages"]
    user_question = messages[-1].content

    # Construct prompt with user's past questions
    past_interactions = "\n".join(
        [msg.content for msg in messages if isinstance(msg, HumanMessage)]
    )

    prompt_content = constants.AGENT_PROMPT.format(past_interactions=past_interactions, user_question=user_question)
    model = llm.bind_tools(tools)
    response = model.invoke([HumanMessage(content=prompt_content)])
    # We return a list, because this will get added to the existing list
    return {"messages": [response], "user_question": user_question}

def rewrite(state):
    """
    Transform the query to produce a better question.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with re-phrased question
    """

    print("---TRANSFORM QUERY---")
    # messages = state["messages"]
    question = state["user_question"]

    msg = [
        HumanMessage(
            content=constants.REWRITE_QUERY.format(question=question),
        )
    ]

    # Grader
    model = llm
    response = model.invoke(msg)
    return {"messages": [response]}


def generate(state):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    messages = state["messages"]
    question = state["user_question"]
    last_message = messages[-1]

    docs = last_message.content
    # .format(question=question, context=docs)
    # Prompt
    prompt = PromptTemplate(
        template=constants.RAG_PROMPT,
        input_variables=["context", "question"]
    )

    # LLM
    model = llm

    # Chain
    rag_chain = prompt | model | StrOutputParser()

    # Run
    response = rag_chain.invoke({"context": docs, "question": question})
    return {"messages": [response]}

In [ ]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # Re-writing the question
workflow.add_node(
    "generate", generate
)  # Generating a response after we know the documents are relevant
# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "agents"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()